In [356]:
#un système d'appariement question-réponse basé sur Tf_idf (question- requête ; reponse,- document indexé par le tf*idf de chaque mot
#dans ce document)
#nous calculons le taux de bon appariement avec cette méthode
# alors, maintenant, nous réfléchissons comment on peut améliorer ce système avec nos termes abtenus

In [357]:
import re
import glob
import json
import string
import operator

In [358]:
from numpy import dot
from numpy.linalg import norm

In [359]:
from nltk.corpus import stopwords

In [360]:
def ouvrir_json(chemin):
    f = open(chemin, encoding="utf−8") 
    toto = json.load(f)
    f.close()
    return toto
def ecrire_json(dataset,chemin):
    donnee = json.dumps(dataset, indent =2,ensure_ascii=False) 
    w = open(chemin, "w",encoding = "utf-8")
    w.write(donnee)
    w.close()
    print("Jeu de données stocké dans %s"%chemin)

In [361]:
#les étapes du prétaitement du corpus 

# listes des tokens à gérer ou à supprimer
listeCaracteres={",":" ", "(":" ", ")":" ","\n":" ", "€":" ","=":" ", "+":" ",
                 "{":" ", "}":" ", "\\":" ", "\"":" ", "«": " ", "»":" ", "%":" ", ";":" ",
                 "/":" ", "_":" ", ": ":" ", " : ":" ",
                 "''":" "}
listeApostropheGerable={"d'":"d' ", "t'":"t' ", "j'":"j' ", "m'":"m' ", "s'":"s' ",
                        "l'":"l' ", "n'":"n' ", "qu'": "qu' ", "D'":"D' ", "T'":"T' ",
                        "J'":"J' ","M'":"M' ", "S'":"S' ", "L'":"L' ", "N'":"N' ", "Qu'": "Qu' "}
listePronomsInverses={"-je":" je", "-tu":" tu", "-il":" il", "-elle":" elle", "-on":" on", "-nous":" nous","-vous":" vous",
                      "-ils":" ils", "-elles":" elles", "-moi":" moi", "-ce":" ce"}
listePointsFinaux=[".", " .", " . ", "!", " !", " ! ",
                   "?", " ?", " ? "]
listeFourreTout={"-t":" t ", "-là":" là ", "@":" ", "$":""}
listeCar={"-":"", ":":""}

def cleanTextFromList(texte, liste):
    for punc, substitute in liste.items():
        texte=texte.replace(punc, substitute)
    return texte

def remplacementListe(listeCar, listeMots):
    newMots=[]
    for mot in listeMots:
        if mot[0] in listeCar.keys():
            substitute = listeCar[mot[0]]
            new=mot.replace(mot[0], substitute)
            newMots.append(new)
        if mot==mot.upper():
            mot = mot.lower()
            newMots.append(mot)
        else:
            newMots.append(mot)
    return(newMots)

In [362]:
#une fonction qui vise à gérer les mots capitalisés au début de la phrase
def gestionMajusculeDebutPhrase(liste):
    if len(liste)>0:
        PremierMot=liste[0]
        PremierMot=PremierMot.lower()
        liste[0]=PremierMot
        for point in listePointsFinaux:
            for i in range(len(liste)):
                if liste[i-1]==point:
                    motAModifier=liste[i]
                    motAModifier=motAModifier.lower()
                    liste[i]=motAModifier
    else:
        liste = liste
    return liste

def suppressionCaracteres(liste, listeCaractere):
    for caractere in listeCaractere:
        for mot in liste :
            if mot==caractere:
                liste.remove(mot)
            if mot=="":
                liste.remove(mot) 
    return liste
listeCar={"-":"", ":":""}

def remplacementListe(listeCar, listeMots):
    newMots=[]
    for mot in listeMots:
        if mot[0] in listeCar.keys():
            substitute = listeCar[mot[0]]
            new=mot.replace(mot[0], substitute)
            newMots.append(new)
        if mot==mot.upper():
            mot = mot.lower()
            newMots.append(mot)
        else:
            newMots.append(mot)
    return(newMots)

In [363]:
import re
def enlever_nombres(texte):
    texte = re.sub("[0-9\.]*", "", texte)
    return texte

In [364]:
def recupArticle_non_cap(texte):
    metaMots = [ ]
    texte=cleanTextFromList(texte, listeCaracteres)
    texte=cleanTextFromList(texte, listeApostropheGerable)
    texte=cleanTextFromList(texte, listePronomsInverses)
    texte=cleanTextFromList(texte, listeFourreTout)
    texte=enlever_nombres(texte)
    mots=texte.split()
    mots=gestionMajusculeDebutPhrase(mots)
    mots=suppressionCaracteres(mots, listePointsFinaux)
    mots=remplacementListe(listeCar, mots)
    for mot in mots:
        if mot.lower( ) not in stopwords.words("french"):
            metaMots.append(mot)
    return metaMots

In [365]:
#lemmatiseur qui ne transforme pas les mots en majuscul
def recupLemmatiseurGLAFF_1(cheminGLAFF):
    lemmatiseur={}
    with open(cheminGLAFF , "r", encoding="utf−8") as GLAFF:
        ligne=GLAFF.readline()
        while ligne:
            elems=re.split("\|", ligne)
            forme=elems[0]
            #forme=forme.capitalize()
            lemme=elems[2]
            #lemme=lemme.capitalize()
            freq=elems[6]
            lemmatiseur[forme]={"lemme":lemme , "freq":float(freq)}
            ligne=GLAFF.readline()
    GLAFF.close()
    return lemmatiseur

In [366]:
def rechercheCorrespondances(mots, lemmatiseur):
    listeLemmes=[]
    for mot in mots:
        lemme=lemmatiseur.get(mot)
        if (lemme):
            listeLemmes.append(lemme["lemme"])
        else:
            listeLemmes.append(mot)
    return listeLemmes

In [367]:
def get_mot_nor(liste):
    mots =  recupArticle_non_cap(liste)
    mots =  rechercheCorrespondances(mots,lemmatiseurGLAFF)
    return mots

In [276]:
lemmatiseurGLAFF = ouvrir_json("corpus/lem/lemGlaff.json")

In [368]:
Q_R ={ }
for chemin in  glob.glob('corpus/QA\*.*')  :
    #print(chemin)
    corpus_q_r = ouvrir_json(chemin)
    for k,v in corpus_q_r.items():
        Q = v[0]
        A=v[1]
        Q_R[Q] = A
        

In [369]:
Q_liste = list(Q_R.keys( ))
R_liste = list(Q_R.values( ))
#print(len(Q_liste))

In [405]:
def norma_q_a(liste):
    n = 0 
    dic_nor = { }
    while n<len(liste):
        dic_nor[n] = get_mot_nor(liste[n])
        n=n+1
    return dic_nor

In [406]:
def get_vocabulaire(articles): #obtenir un vocabulaire propre de nos documents de base (des réponses)
    vocabulaire = [ ]
    for keys, values in articles.items():
        for mot in values:
            if mot not in vocabulaire:
                vocabulaire.append(mot)
    return(vocabulaire)

In [407]:
def get_index(articles):  #obtenir un index qui qui associe un mot vers une liste de documents contenant ce mot 
    vocabulaire = get_vocabulaire(articles)
    myindex = { }
    for m in set(vocabulaire):
        if not m in myindex:
            myindex[m]=[]
        for d in articles:
            if m in articles[d]:
                myindex[m].append(d)
    return myindex

In [408]:
import math
def get_Idf(Index_mots,articles):  #on obtient le IDF de chaque mot
    dic_Idf={}
    for keys, values in Index_mots.items():
        dic_Idf[keys] = math.log10(len(articles)/len(values))
    return dic_Idf
def get_Tf(articles):  #on obtien le TF de chaque mot dans chaque document
    dic_Tf = { }
    for k,v in articles.items():
        wordcount={}
        for mot in v:
            if mot not in wordcount:
                wordcount[mot] = v.count(mot) /len(v)
        dic_Tf[k]= wordcount
    return(dic_Tf)

In [409]:
def get_tf_idf(dic_tf,dic_idf): #on obtient le score de TF*IDF pour chaque mot de chaque q
    dic_tf_idf = {}
    for k, v in dic_tf.items( ):
        for m,tf in v.items():
            tf_idf = tf * dic_idf[m]
            v[m]=tf_idf
        dic_tf_idf[k] = v
    return dic_tf_idf #  {id_1 : [mot1 : tf*idf, mot2 : tif*idf....]...}

In [410]:
def get_vecteur_req(req): # on utilise des chiffres(le TF*IDF de chaque mot) pour représenter la requête
    req = get_mot_nor(req)
    req_vecteur = [ ]
    req_tf = { }
    for mot in req:
        if mot not in req_tf :
            req_tf[mot] =  req.count(mot) /len(req)
    #print(req_tf)
    for mot in req:
        #print(mot)
        if mot in vec :
            mot_tf_idf = req_tf[mot]*Idf_R[mot]
            req_vecteur.append(mot_tf_idf)
        else:
            req_vecteur.append(0)
    return req_vecteur

In [431]:
def get_vecteur_doc(req,liste_id,articles):# on utilise des chiffres(le TF*IDF de chaque mot de requête dans chaque q) pour représenter chaque article
    doc_vecteur = { }# liste_id : la liste des q qui contiennent un (ou plusieurs) mot de la requête  
    req = get_mot_nor(req)
    for id_q in liste_id:
        vecteur = [ ]
        #mots = mots_tf_idf[id_art]
        for m in req:
            #vecteur = [ ]
            if m in articles[id_q]:
                m_tf_idf = Tf_Idf_R[id_q][m]
            else:
                m_tf_idf = 0
            vecteur.append(m_tf_idf)
        if len(vecteur) != vecteur.count(0):
            doc_vecteur[id_q] = vecteur
    return doc_vecteur

In [432]:
def get_m_pre_dic (dic,m):
    n = 0
    dic_m = { }
    for k,v in dic.items():
        dic_m[k] =v
        n = n+1
        if n ==m:
            break
    return dic_m

In [433]:
def recherche_corre_q(vecteur_req,vecteur_doc): 
    ranking =  { }
    for k,v in vecteur_doc.items():
        cos_sim = dot(vecteur_req,v)/(norm(vecteur_req)*norm(v)) #modèl pour calculer le taux de similarité
        ranking[k]=cos_sim   # dictionnaire {id : taux de similarité}
    sorted_ranking={k:v for k,v in sorted(ranking.items (), key=lambda i:i[1], reverse=True )}#mettre le ranking en ordre
    q_corres = get_m_pre_dic(sorted_ranking,1) #q correspondant et probabilité
    return q_corres

In [434]:
def recher_reponse(q_corres,R_liste):
    dic = { }
    for q,pro in q_corres.items( ):
        reponse = R_liste[q]
    dic[reponse] = pro
    print("Reponse : ", reponse)
    print("Probabilité : ", pro)
    return dic

In [435]:
R_nor = norma_q_a(R_liste)
#print(len(R_nor))

In [436]:
vec = get_vocabulaire(R_nor)
index = get_index(R_nor)
Idf_R = get_Idf(index,R_nor)
Tf_R = get_Tf(R_nor)
Tf_Idf_R = get_tf_idf(Tf_R,Idf_R)



In [442]:
#Test 
req = " Comment prendre rendez-vous pour mon inscription administrative ? "
liste_id = list(R_nor.keys( ))
vec_req =  get_vecteur_req(req)
print(vec_req)
vec_doc = get_vecteur_doc(req,liste_id,R_nor)
print(vec_doc)
resul = recherche_corre_q(vec_req,vec_doc)
print(resul)
reponse = recher_reponse(resul,R_liste)

[0.31206104864419226, 0.2384656915852734, 0.2870733013225323, 0.17825969245247714, 0]
{1: [0, 0, 0.027082386917220022, 0, 0], 2: [0, 0.027728568788985275, 0, 0, 0], 3: [0, 0.01064578980291399, 0, 0, 0], 4: [0, 0.056777545615541276, 0, 0, 0], 5: [0, 0.03506848405665785, 0, 0, 0], 6: [0, 0, 0.028144441306130613, 0, 0], 9: [0, 0, 0, 0.022282461556559642, 0], 10: [0, 0, 0, 0.02583473803659089, 0], 12: [0, 0, 0, 0.0557061538913991, 0], 14: [0, 0.03222509345746938, 0.11638106810372931, 0.07226744288613939, 0], 15: [0, 0, 0, 0.018189764535967053, 0], 17: [0.05572518725789147, 0, 0, 0.031832087937942344, 0], 19: [0, 0, 0, 0.019806632494719683, 0], 20: [0, 0, 0, 0.05242932130955209, 0], 21: [0, 0, 0, 0.024089147628713125, 0], 22: [0.022613119466970453, 0, 0, 0.012917369018295445, 0], 23: [0, 0, 0, 0.02970994874207952, 0], 29: [0, 0, 0, 0.07750421410977267, 0], 31: [0, 0, 0, 0.037137435927599396, 0], 54: [0, 0, 0, 0.037137435927599396, 0], 89: [0, 0, 0.05126308952188076, 0, 0], 97: [0, 0.0567775

In [420]:
def apparie_q_r(q_dic):
    dic_apparie = { }
    for id_, q in q_dic.items( ):
        vec_q = get_vecteur_req(q)
        vec_doc = get_vecteur_doc(q,liste_id,R_nor)
        id_corre = list(recherche_corre_q(vec_q,vec_doc).keys( ))[0]
        dic_apparie[id_] = id_corre
    return dic_apparie

In [421]:
q_dic = { }
n=0
while n < len(Q_liste):
    q_dic[n]=Q_liste[n]
    n=n+1

In [422]:
resultat = apparie_q_r(q_dic)
print(resultat)

{0: 99, 1: 1, 2: 6, 3: 99, 4: 2, 5: 6, 6: 6, 7: 10, 8: 9, 9: 8, 10: 22, 11: 38, 12: 12, 13: 13, 14: 14, 15: 9, 16: 16, 17: 10, 18: 18, 19: 21, 20: 21, 21: 21, 22: 22, 23: 23, 24: 99, 25: 31, 26: 45, 27: 90, 28: 28, 29: 30, 30: 30, 31: 22, 32: 30, 33: 3, 34: 30, 35: 12, 36: 37, 37: 37, 38: 44, 39: 10, 40: 107, 41: 8, 42: 42, 43: 42, 44: 75, 45: 9, 46: 21, 47: 96, 48: 50, 49: 107, 50: 107, 51: 78, 52: 99, 53: 56, 54: 99, 55: 99, 56: 22, 57: 99, 58: 107, 59: 64, 60: 10, 61: 75, 62: 99, 63: 49, 64: 64, 65: 1, 66: 107, 67: 52, 68: 9, 69: 10, 70: 102, 71: 107, 72: 3, 73: 39, 74: 75, 75: 75, 76: 15, 77: 77, 78: 107, 79: 107, 80: 70, 81: 107, 82: 75, 83: 58, 84: 52, 85: 107, 86: 107, 87: 19, 88: 64, 89: 64, 90: 49, 91: 75, 92: 92, 93: 107, 94: 96, 95: 107, 96: 78, 97: 17, 98: 99, 99: 1, 100: 27, 101: 9, 102: 75, 103: 42, 104: 9, 105: 99, 106: 107, 107: 99, 108: 108}


In [423]:
nb_corre = 0
for k,v in resultat.items( ):
    if k == v:
        nb_corre = nb_corre+1
        
taux_corre = nb_corre/len(list((resultat.keys())))
print("taux_de_correction :", taux_corre)

taux_de_correction : 0.1743119266055046
